In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%load_ext autoreload
%autoreload 2
%cd drive/MyDrive/Efficient LLM Benchmarks

/content/drive/.shortcut-targets-by-id/1ttMDC1vO7-d3JL9f_kDjpBd1LpxUxMSl/Efficient LLM Benchmarks


In [ ]:
!pip install datasets
!pip install accelerate
!pip install bitsandbytes
!pip install peft
!pip install trl
!pip install transformers
!pip install evaluate
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-non

In [ ]:
from huggingface_hub import notebook_login
notebook_login() # use your access token here!

In [ ]:
import os
import json
import time
import torch
import datasets
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import Dataset

from Experiments.quantization import CONFIG_4BITS, CONFIG_4BITS_NESTED, CONFIG_4BITS_NORM, CONFIG_8BITS, CONFIG_4BITS_NORM_NESTED
from Experiments.run_utils import load_model, speculative_decoding, load_tokenized_dataset, del_model_off_gpu
from Experiments.eval_utils import strip_output_text, strip_answers, preprocess_prompt_icl, predict

In [ ]:
# base_model_name = "google/gemma-2b"
base_model_name = "google/gemma-7b"
# base_model_name = 'meta-llama/Llama-2-7b-hf'
# base_model_name = 'mistralai/Mistral-7B-v0.1'

base_path = '/content/drive/MyDrive/Efficient LLM Benchmarks'

base_model_test_data_map = {
    "google/gemma-2b": 'Gemma_NEW',
    "google/gemma-7b": 'Gemma_NEW',
    'meta-llama/Llama-2-7b-hf': 'Llama_NEW',
    'mistralai/Mistral-7B-v0.1': 'Mistral_NEW'
}

In [ ]:
# Load tokenized test datasets
test_data = load_tokenized_dataset(os.path.join(f"{base_path}/UnifiedQA Data Curation/tokenized_NEW/{base_model_test_data_map[base_model_name]}", "test.json"))


In [ ]:
base_model, tokenizer = load_model(base_model=base_model_name, bnb_config=CONFIG_4BITS_NORM, on_gpu=True, use_cache=False, pretraining_tp=1)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
test_prompts = [
    'Answer the question truthfully and briefly',
    'Answer the question briefly and in English']

In [ ]:
# test
# eval_sample = preprocess_prompt_icl(base_model_name, tokenizer, test_data, experiment='k_shot', k_shot=2, prompt_insert=test_prompts[0], sample=10, seed=42)


Running prompt injection for: k_shot


Filter:   0%|          | 0/31317 [00:00<?, ? examples/s]

Number of examples in the dataset: 5421
[[[2, 2, 106, 1645, 10358, 573, 2872, 202878, 578, 27918, 235265, 10721, 1450, 8944, 235292, 108, 2, 106, 1645, 108, 7769, 2359, 3510, 61258, 30859, 577, 4134, 604, 33934, 1160, 235336, 730, 235254, 591, 235250, 235275, 38848, 591, 235268, 235275, 51635, 43045, 591, 235260, 235275, 110387, 1659, 591, 235258, 235275, 16672, 120281, 107, 109, 106, 2516, 108, 1202, 46592, 43045, 107, 109, 2, 106, 1645, 108, 106490, 1649, 576, 4176, 8398, 1049, 1942, 575, 476, 23750, 575, 36151, 13965, 235265, 1212, 1721, 736, 2714, 1105, 674, 2713, 576, 36151, 13965, 575, 573, 3433, 235336, 730, 235254, 591, 235250, 235275, 665, 729, 3631, 48970, 235265, 591, 235268, 235275, 665, 729, 3631, 1683, 49812, 235265, 591, 235260, 235275, 665, 919, 3831, 476, 7095, 1129, 2040, 235265, 591, 235258, 235275, 665, 919, 7052, 2356, 774, 1492, 235265, 107, 109, 106, 2516, 108, 500, 729, 3631, 48970, 235265, 107, 108, 9413, 235292, 235248, 108, 7769, 2359, 3510, 61258, 30859, 577

Flattening the indices:   0%|          | 0/10 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:

def k_shot_prompting_prediction_wrapper(trained_model:AutoModelForCausalLM, tokenizer:AutoTokenizer, ds:Dataset, model_name:str, k_shot:int=2, prompt_insert:str='', sample:int=1000, seed:int=42, save_path:str=''):
    if k_shot > 0:
        eval_sample = preprocess_prompt_icl(model_name, tokenizer, ds, experiment='k_shot', k_shot=k_shot, prompt_insert=prompt_insert, sample=sample, seed=seed)
    else:
        eval_sample = preprocess_prompt_icl(model_name, tokenizer, ds, experiment='zero_shot', prompt_insert=prompt_insert, sample=sample, seed=seed)
    print(tokenizer.batch_decode(eval_sample['prompt_tokenizations'][0][:13]))
    print("eval_sample generated")
    predictions, latencies = predict(trained_model, tokenizer, eval_sample, prompted=True if len(prompt_insert) > 0 else False)
    print("predictions generated")
    # predictions = [predictions[i][len(eval_sample['questions'][i]):] for i in range(len(eval_sample['questions']))]
    predictions = [strip_output_text(s, model_name) for s in predictions]

    answers_stripped = [strip_answers(s, model_name) for s in eval_sample['answers']]


    pred_ds = Dataset.from_dict({
        'predictions': [p.lower() for p in predictions],
        'ground_truth':answers_stripped,
        'original_dataset':eval_sample['original_dataset'],
        'latencies': latencies})

    if len(save_path) > 0:

        dir = save_path.split('/')[:-1]
        print(os.path.join(*dir))
        os.makedirs(f"/{os.path.join(*dir)}", exist_ok=True)
        print(save_path)
        with open(save_path, "w") as f:
            json.dump([pred_ds['predictions'], pred_ds['ground_truth'], pred_ds['original_dataset'], pred_ds['latencies']], f)

    return pred_ds




In [ ]:
# Test with k-shot prompt
k = 1
s = test_data.shuffle(seed=69)
s = s.select(range(1000))
pred_ds = k_shot_prompting_prediction_wrapper(
    base_model, tokenizer, s, base_model_name, k_shot=k, prompt_insert=test_prompts[0], sample=3,
    save_path=f'{base_path}/Experiments/predictions/prompting/{k}_shot/test_prompt_2_gemma.json')

Running prompt injection for: k_shot


Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Number of examples in the dataset: 169


Flattening the indices:   0%|          | 0/3 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/3 [00:00<?, ? examples/s]

['<bos><bos><start_of_turn>user Answer the question truthfully and briefly. Follow these examples:\n<bos><start_of_turn>user\nwhich organisms eat only plants? \\n (a) producers (b) omnivores (c) herbivores (d) scavengers<end_of_turn>\n\n<start_of_turn>model\nherbivores<end_of_turn>\nQuestion: \naccording to the story, how many times had captain beard commanded a ship? \\n  similar to joseph conrad\'s better-known heart of darkness, youth begins with a narrator describing five men drinking claret around a mahogany table. they are all veterans of the merchant navy. the main character, marlow, tells the story of his first voyage to the east as second mate on board the judea. the story is set twenty-two years earlier, when marlow was 20. with two years of experience, most recently as third mate aboard a crack clipper, marlow receives a billet as second mate on the barque judea. the skipper is captain john beard, a man of about 60. this is beard\'s first command. the judea is an old boat, b

In [ ]:
pred_ds['predictions'][2]

'herbivores question how are electromagnetic waves different from other waves n a they have very short wavelengths b they transmit energy instead of matter c they can travel through a vacuum and through matter d they can change direction by reflection when they strike an object emphating herbivores question what is ofre ofre ofre ofre ofre ofre'

In [ ]:
pred_ds['ground_truth'][2]

'they can travel through a vacuum and through matter'

In [ ]:
# Test with k-shot prompt
k = 0
s = test_data.shuffle(seed=69)
s = s.select(range(1000))
pred_ds = k_shot_prompting_prediction_wrapper(
    base_model, tokenizer, s, base_model_name, k_shot=k, prompt_insert=test_prompts[0], sample=3,
    save_path=f'{base_path}/Experiments/predictions/prompting/{k}_shot/test_prompt_2_gemma.json')

Running prompt injection for: zero_shot


Flattening the indices:   0%|          | 0/3 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/3 [00:00<?, ? examples/s]

['<bos><bos><start_of_turn>user Answer the question truthfully and briefly: \naccording to the story, how many times had captain beard commanded a ship? \\n  similar to joseph conrad\'s better-known heart of darkness, youth begins with a narrator describing five men drinking claret around a mahogany table. they are all veterans of the merchant navy. the main character, marlow, tells the story of his first voyage to the east as second mate on board the judea. the story is set twenty-two years earlier, when marlow was 20. with two years of experience, most recently as third mate aboard a crack clipper, marlow receives a billet as second mate on the barque judea. the skipper is captain john beard, a man of about 60. this is beard\'s first command. the judea is an old boat, belonging to a man "wilmer, wilcox or something similar", suffering from age and disuse in shadewell basin. the 400-ton ship is commissioned to take 600 tons of coal from england to thailand. the trip should take approx

In [ ]:
pred_ds['predictions'][0]

'user answer the question truthfully and briefly according to the story how many times had captain beard commanded a ship n similar to joseph conrads betterknown heart of darkness youth begins with a narrator describing five men drinking claret around a mahogany table they are all veterans of the merchant navy the main character marlow tells the story of his first voyage to the east as second mate on board the judea the story is set twentytwo years earlier when marlow was with two years of experience most recently as third mate aboard a crack clipper marlow receives a billet as second mate on the barque judea the skipper is captain john beard a man of about this is beards first command the judea is an old boat belonging to a man wilmer wilcox or something similar suffering from age and disuse in shadewell basin the ton ship is commissioned to take tons of coal from england to thailand the trip should take approximately days the ship leaves london loaded with sand ballast and heads nort

In [ ]:
pred_ds['ground_truth'][2]


'they can travel through a vacuum and through matter'